In [18]:
import os
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [19]:
# 로컬 경로 설정 (본인 환경에 맞게 수정)
base_path = "D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/"
handle_path = os.path.join(base_path, "exolyt_result2.csv")
result_csv_path = os.path.join(base_path, "exolyt_result3.csv")

# # 엑셀에서 Username 추출
# handle_df = pd.read_excel(handle_path)
# usernames = handle_df.iloc[:, 0].dropna().apply(lambda x: x.lstrip('@')).tolist()

# print("✅ 사용자 수:", len(usernames))
# print("🎯 미리보기:", usernames[:])

In [20]:
handle_df = pd.read_csv(handle_path)
handle_df

handle_df = handle_df[handle_df['last_video_date'].isna()]
handle_df

,username,last_video_date
13,ugc_withjessi,NaN
15,jessicaroseugc,NaN
34,allbriwoo,NaN
174,sophia.cathyy,NaN
189,abigail.ugc.creator,NaN
...,...,...
285,emileerachel,NaN
286,timeforrice,NaN
287,jhamheartsthis,NaN
288,aglimpseofglow,NaN


In [21]:
usernames = handle_df.iloc[:, 0]

print("✅ 사용자 수:", len(usernames))
print("🎯 미리보기:", usernames[:10])

✅ 사용자 수: 100
🎯 미리보기: 13           ugc_withjessi
15          jessicaroseugc
34               allbriwoo
174          sophia.cathyy
189    abigail.ugc.creator
194        annahostriakova
195               ibubblii
196      beautyfanaticjenn
197           glowyyclover
198          ugcwithedwina
Name: username, dtype: object


In [22]:
import json

def apply_login_cookies(driver, cookie_path):
    with open(cookie_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    cookies_raw = json_data.get('cookies', [])
    
    # 문자열인 쿠키 → 객체로 변환
    cookies = []
    for c in cookies_raw:
        if isinstance(c, str):
            try:
                cookies.append(json.loads(c))
            except:
                continue
        else:
            cookies.append(c)

    # 첫 방문 (쿠키 설정을 위해)
    driver.get("https://www.exolyt.com")
    time.sleep(2)

    for cookie in cookies:
        try:
            cookie.pop('domain', None)  # 도메인 충돌 방지
            driver.add_cookie(cookie)
        except Exception as e:
            print(f"⚠️ 쿠키 추가 실패: {cookie.get('name', 'N/A')} → {e}")


In [23]:
def handle_track_account_popup(driver):
    try:
        # 1️⃣ 첫 번째 팝업: "Track account" 버튼 클릭
        track_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div/div/div/button'))
        )
        track_button.click()
        print("✅ 1단계: 'Track account' 클릭")

        # 2️⃣ 두 번째 팝업: "Great!" 버튼 클릭 (뜨면)
        try:
            great_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[4]/div/div/div/button'))
            )
            great_button.click()
            print("✅ 2단계: 'Great!' 팝업 닫기 완료")
        except:
            print("ℹ️ 'Great!' 팝업 없음")

        # 3️⃣ 다시 한번 'Track account' 클릭 (Analytics 탭 열리게 하기 위해)
        time.sleep(1)
        try:
            track_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div/div/div/button'))
            )
            track_button.click()
            print("✅ 3단계: 'Track account' 두 번째 클릭 완료")
        except:
            print("ℹ️ 3단계에서 Track 버튼 없음 (이미 닫힘 가능성)")
    except:
        print("ℹ️ 'Account data is outdated' 팝업 없음")


In [24]:
def crawl_exolyt(username):
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("start-maximized")
        options.add_argument("user-agent=Mozilla/5.0")

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # ✅ 1. 로그인 쿠키 적용
        cookie_path = os.path.join(base_path, "exolyt_login_state.json")
        driver.get("https://www.exolyt.com")
        time.sleep(2)
        apply_login_cookies(driver, cookie_path)

        # ✅ 2. 유저 페이지 이동
        driver.get(f"https://exolyt.com/user/tiktok/{username}")
        time.sleep(random.uniform(4, 6))

        # ✅ 3. 로딩 완료 대기
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, "ds-fade-in"))
        )

        # ✅ 4. 팝업 닫기 (있으면 닫고, 없으면 통과)
        try:
            close_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[3]/div/svg'))
            )
            driver.execute_script("arguments[0].click();", close_btn)
            print("✅ 팝업 닫기 완료")
            time.sleep(2)
        except Exception:
            print("ℹ️ 팝업 없음 → 그대로 진행")

        # ✅ 5. Overview 클릭
        overview_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Overview')]"))
        )
        driver.execute_script("arguments[0].click();", overview_btn)
        print("✅ Overview 클릭 완료")
        time.sleep(2)

        # ✅ 6. 'Choose date range' 체크박스 클릭 여부 확인
        checkbox_btn = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@type='checkbox' and contains(@class, 'cursor-pointer')]"))
        )
        is_checked = driver.execute_script("return arguments[0].checked;", checkbox_btn)
        if not is_checked:
            driver.execute_script("arguments[0].click();", checkbox_btn)
            print("✅ 'Choose date range' 체크박스 클릭 완료")
        else:
            print("ℹ️ 'Choose date range' 체크박스는 이미 선택되어 있음")
        time.sleep(2)

        # ✅ 7. Last video 날짜 추출 (텍스트 기반)
        try:
            label_td = driver.find_element(By.XPATH, "//td[contains(text(), 'First video')]")
            last_video_date = label_td.find_element(By.XPATH, "following-sibling::td").text
        except Exception as e:
            print(f"❌ Last video 날짜 추출 실패: {e}")
            last_video_date = None

        data = {
            "username": username,
            "last_video_date": last_video_date
        }

        print("📊 최종 수집 데이터:", data)
        return data

    except Exception as e:
        print(f"❌ {username} - 실패: {e}")
        return None

    finally:
        try:
            driver.quit()
        except:
            pass

In [25]:
results = []

for i, username in enumerate(usernames):
    print(f"\n[{i+1}/{len(usernames)}] ▶ {username}")
    result = crawl_exolyt(username)
    if result:
        results.append(result)
        print(f"✅ {username} - 완료")
    else:
        print(f"❌ {username} - 저장되지 않음")


[1/100] ▶ ugc_withjessi
ℹ️ 팝업 없음 → 그대로 진행
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'ugc_withjessi', 'last_video_date': '2022-01-16'}
✅ ugc_withjessi - 완료

[2/100] ▶ jessicaroseugc
ℹ️ 팝업 없음 → 그대로 진행
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'jessicaroseugc', 'last_video_date': '2023-09-30'}
✅ jessicaroseugc - 완료

[3/100] ▶ allbriwoo
ℹ️ 팝업 없음 → 그대로 진행
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'allbriwoo', 'last_video_date': '2023-01-09'}
✅ allbriwoo - 완료

[4/100] ▶ sophia.cathyy
ℹ️ 팝업 없음 → 그대로 진행
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'sophia.cathyy', 'last_video_date': 'N/A'}
✅ sophia.cathyy - 완료

[5/100] ▶ abigail.ugc.creator
ℹ️ 팝업 없음 → 그대로 진행
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'abigail.ugc.creator', 'last_video_date': 'N/A'}
✅ abigail.ugc.creator - 완료

[6/100] ▶ annahostriakova
ℹ️ 팝업 없음 → 그대로 진행
✅ Overview

In [26]:
result_df = pd.DataFrame(results)
result_df.to_csv(result_csv_path, index=False, encoding='utf-8-sig')
print(f"\n📁 저장 완료: {result_csv_path}")


📁 저장 완료: D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/exolyt_result3.csv
